# 1. ETL (Carga y transformación de datos)

Importamos las líbrerias a utilizar

In [1]:
import pandas as pd
import json # Para trabajar con JSON
from pandas import json_normalize
pd.set_option('display.max_columns', None)
# Recarga automática de módulos antes de ejecutar
#%load_ext autoreload
#%autoreload 2

import warnings
warnings.filterwarnings("ignore")


## 1.1. Ingesta de datos (Extraction)

Se cuenta con dos archivos csv 

In [2]:
# Cargamos el df1
df1=pd.read_csv(r'D:/2024/HenryData/Py_Individual/datos_crudos/movies_dataset.csv', encoding='utf-8')

In [3]:
print(df1.shape)
df1.head(2)

(45466, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [5]:
# Para unir los dfs necesitamos cambiar el tipo de datos del 'id' del df1
df1 = df1[df1['id'].str.isnumeric()] #Eliminamos las filas que contengan valores nnúmericos
#Cambiamos el tipo de dato 
df1['id'] = df1['id'].astype(int)

In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45463 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45463 non-null  object 
 1   belongs_to_collection  4491 non-null   object 
 2   budget                 45463 non-null  object 
 3   genres                 45463 non-null  object 
 4   homepage               7779 non-null   object 
 5   id                     45463 non-null  int32  
 6   imdb_id                45446 non-null  object 
 7   original_language      45452 non-null  object 
 8   original_title         45463 non-null  object 
 9   overview               44509 non-null  object 
 10  popularity             45460 non-null  object 
 11  poster_path            45077 non-null  object 
 12  production_companies   45460 non-null  object 
 13  production_countries   45460 non-null  object 
 14  release_date           45376 non-null  object 
 15  revenue

In [7]:
# Cargamos el df2
df2=pd.read_csv(r'D:/2024/HenryData/Py_Individual/datos_crudos/credits.csv', encoding='utf-8')
#D:\2024\HenryData\Py_Individual\datos_crudos\credits.csv

In [8]:
print(df2.shape)
df2.head(2)

(45476, 3)


,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844


In [9]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [10]:
# Unimos los df1 y df2 sólo las filas que coincidan en 'id' en ambos
df_movies = pd.merge(df1, df2, on='id', how='inner')


In [11]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45538 entries, 0 to 45537
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45538 non-null  object 
 1   belongs_to_collection  4500 non-null   object 
 2   budget                 45538 non-null  object 
 3   genres                 45538 non-null  object 
 4   homepage               7792 non-null   object 
 5   id                     45538 non-null  int32  
 6   imdb_id                45521 non-null  object 
 7   original_language      45527 non-null  object 
 8   original_title         45538 non-null  object 
 9   overview               44584 non-null  object 
 10  popularity             45535 non-null  object 
 11  poster_path            45152 non-null  object 
 12  production_companies   45535 non-null  object 
 13  production_countries   45535 non-null  object 
 14  release_date           45451 non-null  object 
 15  re

## 2. Tratamiento de datos (Transform): 

In [12]:
print(df_movies.shape)
df_movies.head(2)

(45538, 26)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."


Eliminamos columnas que no se necesitaran

In [13]:
column_drop=['video','imdb_id','adult','original_title','poster_path','homepage'] # columnas a eliminar


In [14]:
df_movies= df_movies.drop(column_drop, axis=1)

In [15]:
# Luego de eliminar las columnas que no serán necesarias para el análisis se cuentan con 19 columnas.
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45538 entries, 0 to 45537
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4500 non-null   object 
 1   budget                 45538 non-null  object 
 2   genres                 45538 non-null  object 
 3   id                     45538 non-null  int32  
 4   original_language      45527 non-null  object 
 5   overview               44584 non-null  object 
 6   popularity             45535 non-null  object 
 7   production_companies   45535 non-null  object 
 8   production_countries   45535 non-null  object 
 9   release_date           45451 non-null  object 
 10  revenue                45535 non-null  float64
 11  runtime                45278 non-null  float64
 12  spoken_languages       45535 non-null  object 
 13  status                 45454 non-null  object 
 14  tagline                20439 non-null  object 
 15  ti

In [16]:
# Se muesta el DF con las nuevas columnas, se observa que algunas columnas estan anidados en formato de diccionario y listas. 
df_movies.head(2)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."


*Columna anidada:*  belongs_to_collection, los valores de esta columna estan anidadas en formato cadena y las convertiremos a diccionario, para ello se emplea la funcion convert_a_dic, luego aplicamos la función en la columna y finalmente extraer el valor 'name' de la columna.

In [17]:
import ast #importamos la libreria
# Funcion 1, para convertir los valores de la comlumna que estan en formato cadenas a diccionario 
def convert_a_dic(val):
    if isinstance(val, str):  # Verifica si el valor es una cadena
        try:
            return ast.literal_eval(val)  # Intenta convertir la cadena a diccionario
        except (ValueError, SyntaxError):
            return None  # Si falla, devuelve None (para representar sin colección)
    return val  # Si ya es un diccionario, lo retorna sin cambios

In [18]:
# Aplicamos la función para convertir de candena a diccionario
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].apply(convert_a_dic)

In [19]:
# Extraemos el valor name del diccionario y si esta vacío se coloca 'No collecction' para no perder datos
df_movies['belongs_to_collection']=df_movies['belongs_to_collection'].apply(lambda x: x['name']if isinstance(x, dict) else 'No collecction')

In [20]:
df_movies.head(2) # verificamos los cambios

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,Toy Story Collection,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,No collecction,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."


*Columnas anidadas:* genres, production_companies, production_countries, spoken_languages, cast, crew; en el caso de estas columnas los valores estan anidadas en formato cadena y las convertiremos a lista, para ello se emplea la funcion convert_a_list, luego aplicamos la función en cada una de las columnas, con el objetivo de extraer el valor requerido para el análisis de la columna.

In [21]:
# Función 2 para convertir la cadena en lista 
def convert_a_list(val):
    if isinstance(val, str):  # Verifica si el valor es una cadena
        try:
            return ast.literal_eval(val)  # Intenta convertir la cadena a lista
        except (ValueError, SyntaxError):
            return []  # Si falla, devuelve una lista vacía
    elif isinstance(val, (float, bool)) and pd.isna(val):  # Maneja valores nulos (NaN)
        return []
    elif isinstance(val, (float, bool)):  # Maneja casos en los que es un valor booleano o float no nulo
        return []
    return val  # Si ya es una lista, lo retorna sin cambios

In [ ]:
# Aplciamos la funcion para convertir de candena a lista 
df_movies['genres'] = df_movies['genres'].apply(convert_a_list)

# Extraemos el valor name 
df_movies['genres'] = df_movies['genres'].apply(lambda x: [d['name'] for d in x if 'name' in d])

# Para que el valor de la columna género aparezca sepadado por una coma 
df_movies['genres'] = df_movies['genres'].apply(lambda x: ', '.join(x))

In [ ]:
# Aplicamos la funcion para convertir de candena a lista 
df_movies['production_companies'] = df_movies['production_companies'].apply(convert_a_list)
# Extraemos el valor name 
df_movies['production_companies'] = df_movies['production_companies'].apply(lambda x: [d['name'] for d in x if 'name' in d])
# Para que los valores de la columna aparezca separado por una coma
df_movies['production_companies'] = df_movies['production_companies'].apply(lambda x: ' '.join(x))

In [ ]:
# Aplicamos la funcion para convertir de candena a lista 
df_movies['production_countries'] = df_movies['production_countries'].apply(convert_a_list)
# Extraemos el valor name del dic
df_movies['production_countries'] = df_movies['production_countries'].apply(lambda x: [d['name'] for d in x if 'name' in d])
# Para que los valores de la columna aparezca separado por una coma
df_movies['production_countries'] = df_movies['production_countries'].apply(lambda x: ', '.join(x))

In [ ]:
# Aplicamos la funcion para convertir de candena a lista
df_movies['spoken_languages'] = df_movies['spoken_languages'].apply(convert_a_list)
# Extraemos el valor name 
df_movies['spoken_languages'] = df_movies['spoken_languages'].apply(lambda x: [d['name'] for d in x if 'name' in d])
# Para que los valores de la columna aparezca separado por una coma
df_movies['spoken_languages'] = df_movies['spoken_languages'].apply(lambda x: ', '.join(x))

In [26]:
# Observamos los cambios en el DF
df_movies.head(2)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,Toy Story Collection,30000000,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,English,Released,NaN,Toy Story,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,No collecction,65000000,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures Teitler Film Interscope Commu...,United States of America,1995-12-15,262797249.0,104.0,"English, Français",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."


In [27]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45538 entries, 0 to 45537
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  45538 non-null  object 
 1   budget                 45538 non-null  object 
 2   genres                 45538 non-null  object 
 3   id                     45538 non-null  int32  
 4   original_language      45527 non-null  object 
 5   overview               44584 non-null  object 
 6   popularity             45535 non-null  object 
 7   production_companies   45538 non-null  object 
 8   production_countries   45538 non-null  object 
 9   release_date           45451 non-null  object 
 10  revenue                45535 non-null  float64
 11  runtime                45278 non-null  float64
 12  spoken_languages       45538 non-null  object 
 13  status                 45454 non-null  object 
 14  tagline                20439 non-null  object 
 15  ti

Modificar: Tipo de dato de la columna 'budget' y completar las filas que tenhan NAN con cero 

In [28]:
# Observamos que la colunna budget es de tipo objeto, por lo que se debe de convertir en númerico
df_movies['budget'] = pd.to_numeric(df_movies['budget'], errors='coerce')

In [29]:
# Rellenar los NaN de la columna 'budget'con 0 
df_movies['budget'].fillna(0, inplace=True)  

In [30]:
# Observamos los cambios
df_movies.head(2)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,Toy Story Collection,30000000,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,English,Released,NaN,Toy Story,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,No collecction,65000000,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures Teitler Film Interscope Commu...,United States of America,1995-12-15,262797249.0,104.0,"English, Français",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."


In [31]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45538 entries, 0 to 45537
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  45538 non-null  object 
 1   budget                 45538 non-null  int64  
 2   genres                 45538 non-null  object 
 3   id                     45538 non-null  int32  
 4   original_language      45527 non-null  object 
 5   overview               44584 non-null  object 
 6   popularity             45535 non-null  object 
 7   production_companies   45538 non-null  object 
 8   production_countries   45538 non-null  object 
 9   release_date           45451 non-null  object 
 10  revenue                45535 non-null  float64
 11  runtime                45278 non-null  float64
 12  spoken_languages       45538 non-null  object 
 13  status                 45454 non-null  object 
 14  tagline                20439 non-null  object 
 15  ti

Eliminar duplicados según 'id'

In [32]:
# Para la columna id verificamos si hay duplicados 
duplica = df_movies[df_movies.duplicated(['id'])]


In [33]:
print(f"Duplicados encontrados: {len(duplica)}")

Duplicados encontrados: 106


In [34]:
# Eliminamos las filas del id duplicado
df_movies = df_movies.drop_duplicates(subset='id', keep='first') # first significa que mantenedremos la primera aparicion del id y eliminamos las siguientes

In [35]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45432 entries, 0 to 45537
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  45432 non-null  object 
 1   budget                 45432 non-null  int64  
 2   genres                 45432 non-null  object 
 3   id                     45432 non-null  int32  
 4   original_language      45421 non-null  object 
 5   overview               44478 non-null  object 
 6   popularity             45429 non-null  object 
 7   production_companies   45432 non-null  object 
 8   production_countries   45432 non-null  object 
 9   release_date           45345 non-null  object 
 10  revenue                45429 non-null  float64
 11  runtime                45172 non-null  float64
 12  spoken_languages       45432 non-null  object 
 13  status                 45348 non-null  object 
 14  tagline                20401 non-null  object 
 15  title  

In [36]:
# Encontrar el idioma más frecuente
most_frequent_language = df_movies['original_language'].mode()[0]

print(most_frequent_language)

en


In [37]:
# Rellenar los nulos
df_movies['original_language'] = df_movies['original_language'].fillna(most_frequent_language)

In [38]:
df_movies.head(2)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,Toy Story Collection,30000000,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,English,Released,NaN,Toy Story,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,No collecction,65000000,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures Teitler Film Interscope Commu...,United States of America,1995-12-15,262797249.0,104.0,"English, Français",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."


Columna 'release_date': Se elimina valores nulos, se convierte a tipo de dato datetime y a formato año-mes-día y se crea la columna release_year 

In [39]:
# Valores nulos eliminados del campo release date
df_movies = df_movies.dropna(subset=['release_date'])

In [40]:
# Convertimos la columna 'release_date' a tipo de dato datetime, y a formato año-mes-día
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], errors='coerce')

In [41]:
# Contar valores nulos en 'release_date' después de la conversión
missing_date = df_movies['release_date'].isna().sum()
print(f"Valores nulos en 'release_date' después de la conversión: {missing_date}")

Valores nulos en 'release_date' después de la conversión: 0


In [42]:
# Crearmos la columna release_year en base a 'release date'
df_movies['release_year'] = df_movies['release_date'].dt.year

In [43]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45345 entries, 0 to 45537
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  45345 non-null  object        
 1   budget                 45345 non-null  int64         
 2   genres                 45345 non-null  object        
 3   id                     45345 non-null  int32         
 4   original_language      45345 non-null  object        
 5   overview               44404 non-null  object        
 6   popularity             45345 non-null  object        
 7   production_companies   45345 non-null  object        
 8   production_countries   45345 non-null  object        
 9   release_date           45345 non-null  datetime64[ns]
 10  revenue                45345 non-null  float64       
 11  runtime                45099 non-null  float64       
 12  spoken_languages       45345 non-null  object        
 13  status

In [44]:
# Observamos las modificaciones
df_movies.head(2)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,release_year
0,Toy Story Collection,30000000,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,English,Released,NaN,Toy Story,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",1995
1,No collecction,65000000,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures Teitler Film Interscope Commu...,United States of America,1995-12-15,262797249.0,104.0,"English, Français",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",1995


Columna 'overview': Se rellena los valores nulos con 'No overview'

In [45]:
# Rellenar valores nulos con una 'No overview' en el resumen de la película
df_movies['overview'] = df_movies['overview'].fillna('No overview')

In [46]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45345 entries, 0 to 45537
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  45345 non-null  object        
 1   budget                 45345 non-null  int64         
 2   genres                 45345 non-null  object        
 3   id                     45345 non-null  int32         
 4   original_language      45345 non-null  object        
 5   overview               45345 non-null  object        
 6   popularity             45345 non-null  object        
 7   production_companies   45345 non-null  object        
 8   production_countries   45345 non-null  object        
 9   release_date           45345 non-null  datetime64[ns]
 10  revenue                45345 non-null  float64       
 11  runtime                45099 non-null  float64       
 12  spoken_languages       45345 non-null  object        
 13  status

In [47]:
df_movies.head(2)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,release_year
0,Toy Story Collection,30000000,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,English,Released,NaN,Toy Story,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",1995
1,No collecction,65000000,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures Teitler Film Interscope Commu...,United States of America,1995-12-15,262797249.0,104.0,"English, Français",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",1995


Columna 'popularity': se modifica el tipo de dato a numérico y los valores NaN se coloca cero.

In [48]:
# La columna 'popularity' es de tipo objeto, debemos de convertirlo a tipo númerico y reemplazar errores con NaN
df_movies['popularity'] = pd.to_numeric(df_movies['popularity'], errors='coerce')

In [49]:
# Contar valores nulos en 'popularity' después de la conversión
missing_popularity = df_movies['popularity'].isna().sum()
print(f"Valores nulos en 'popularity' después de la conversión: {missing_popularity}")

Valores nulos en 'popularity' después de la conversión: 0


In [50]:
# Observamos los cambios
df_movies.head(2)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,release_year
0,Toy Story Collection,30000000,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,English,Released,NaN,Toy Story,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",1995
1,No collecction,65000000,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures Teitler Film Interscope Commu...,United States of America,1995-12-15,262797249.0,104.0,"English, Français",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",1995


In [51]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45345 entries, 0 to 45537
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  45345 non-null  object        
 1   budget                 45345 non-null  int64         
 2   genres                 45345 non-null  object        
 3   id                     45345 non-null  int32         
 4   original_language      45345 non-null  object        
 5   overview               45345 non-null  object        
 6   popularity             45345 non-null  float64       
 7   production_companies   45345 non-null  object        
 8   production_countries   45345 non-null  object        
 9   release_date           45345 non-null  datetime64[ns]
 10  revenue                45345 non-null  float64       
 11  runtime                45099 non-null  float64       
 12  spoken_languages       45345 non-null  object        
 13  status

*Columnas anidadas:* cast, crew; en el caso de estas columnas los valores estan anidadas en formato cadena y las convertiremos a lista, para ello se emplea la funcion convert_a_list, para luego aplicar la función en cada una de las columnas, con el objetivo de extraer el valor requerido para el análisis en una nueva columna, respectivamente las cuales serán actor y director.

In [ ]:
# Aplicamos la funcion para convertir de candena a lista 
df_movies['cast'] = df_movies['cast'].apply(convert_a_list)

# Extraemos el valor name
df_movies['actor'] = df_movies['cast'].apply(lambda x: [d['name'] for d in x[:3] if 'name' in d])

# Para que el valor de la columna aparezca sepadado por una coma
df_movies['actor'] = df_movies['actor'].apply(lambda x: ', '.join(x))

In [ ]:
# Aplicar la función para convertir de cadena a lista 
df_movies['crew'] = df_movies['crew'].apply(convert_a_list)

# Extraer el nombre del director si el trabajo (job) es 'director'
df_movies['director'] = df_movies['crew'].apply(lambda x: [d['name'] for d in x if isinstance(d, dict) and d.get('job') == 'Director'])

# Para que el valor de la columna aparezca sepadado por una coma
df_movies['director'] = df_movies['director'].apply(lambda x: ', '.join(x) if x else None)

In [54]:
df_movies.head(2)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,release_year,actor,director
0,Toy Story Collection,30000000,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,English,Released,NaN,Toy Story,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",1995,"Tom Hanks, Tim Allen, Don Rickles",John Lasseter
1,No collecction,65000000,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures Teitler Film Interscope Commu...,United States of America,1995-12-15,262797249.0,104.0,"English, Français",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",1995,"Robin Williams, Jonathan Hyde, Kirsten Dunst",Joe Johnston


In [55]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45345 entries, 0 to 45537
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  45345 non-null  object        
 1   budget                 45345 non-null  int64         
 2   genres                 45345 non-null  object        
 3   id                     45345 non-null  int32         
 4   original_language      45345 non-null  object        
 5   overview               45345 non-null  object        
 6   popularity             45345 non-null  float64       
 7   production_companies   45345 non-null  object        
 8   production_countries   45345 non-null  object        
 9   release_date           45345 non-null  datetime64[ns]
 10  revenue                45345 non-null  float64       
 11  runtime                45099 non-null  float64       
 12  spoken_languages       45345 non-null  object        
 13  status

Columna 'return': Creamos esta columna en base a las columnas 'revenue' y 'budget', en caso no se tenga valor se tomará el valor cero 

In [56]:
# Crear la columna return, con los campos revenue y buget, si no hay datos tomará el valor 0
df_movies['return'] = df_movies.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] > 0 else 0, axis=1)

In [57]:
# Observamos los cambios
df_movies.head(2)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,release_year,actor,director,return
0,Toy Story Collection,30000000,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,English,Released,NaN,Toy Story,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",1995,"Tom Hanks, Tim Allen, Don Rickles",John Lasseter,12.451801
1,No collecction,65000000,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures Teitler Film Interscope Commu...,United States of America,1995-12-15,262797249.0,104.0,"English, Français",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",1995,"Robin Williams, Jonathan Hyde, Kirsten Dunst",Joe Johnston,4.043035


In [58]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45345 entries, 0 to 45537
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  45345 non-null  object        
 1   budget                 45345 non-null  int64         
 2   genres                 45345 non-null  object        
 3   id                     45345 non-null  int32         
 4   original_language      45345 non-null  object        
 5   overview               45345 non-null  object        
 6   popularity             45345 non-null  float64       
 7   production_companies   45345 non-null  object        
 8   production_countries   45345 non-null  object        
 9   release_date           45345 non-null  datetime64[ns]
 10  revenue                45345 non-null  float64       
 11  runtime                45099 non-null  float64       
 12  spoken_languages       45345 non-null  object        
 13  status

# Filtrado del DataFrame:
Con el objetivo de optimizar la data para el análisis exploratorio, se ha filtrado los datos según los siguientes criterio: idioma original o traducción en  español o inglés de las películas ( columna'spoken_languages'), y año de estreno de las películas posterior a 1980  películas. A continuación se explorar los datos y se realiza el filtrado.

In [59]:
unique_languages = list(df_movies['original_language'].unique())
print(unique_languages)

['en', 'fr', 'zh', 'it', 'fa', 'nl', 'de', 'cn', 'ar', 'es', 'ru', 'sv', 'ja', 'ko', 'sr', 'bn', 'he', 'pt', 'wo', 'ro', 'hu', 'cy', 'vi', 'cs', 'da', 'no', 'nb', 'pl', 'el', 'sh', 'xx', 'mk', 'bo', 'ca', 'fi', 'th', 'sk', 'bs', 'hi', 'tr', 'is', 'ps', 'ab', 'eo', 'ka', 'mn', 'bm', 'zu', 'uk', 'af', 'la', 'et', 'ku', 'fy', 'lv', 'ta', 'sl', 'tl', 'ur', 'rw', 'id', 'bg', 'mr', 'lt', 'kk', 'ms', 'sq', 'qu', 'te', 'am', 'jv', 'tg', 'ml', 'hr', 'lo', 'ay', 'kn', 'eu', 'ne', 'pa', 'ky', 'gl', 'uz', 'sm', 'mt', 'hy', 'iu', 'lb', 'si']


In [60]:
unique_spok = list(df_movies['spoken_languages'].unique())
print(unique_spok)

['English', 'English, Français', 'English, Español', 'Français, English', 'English, Deutsch', 'English, Pусский, Español', 'English, Latin', 'English, Español, Nederlands', '广州话 / 廣州話, Français', '普通话', 'Pусский, English', 'Español, English', 'Español, English, Français, Magyar', '', 'shqip, Italiano', 'Italiano, Español', 'English, Français, 한국어/조선말, Español', 'English, Pусский', 'English, Magyar', 'Français, Español', 'Français', 'Deutsch, English', 'فارسی', 'Nederlands', 'Dansk, English, Français, Deutsch', 'English, Français, Pусский', 'Deutsch', 'English, Français, Latin, ', '广州话 / 廣州話, English', 'English, Français, Deutsch', '广州话 / 廣州話, English, 日本語, 普通话', 'Italiano', 'العربية, Français', 'Español', 'English, Hrvatski, Bosanski', 'Français, , Español', 'English, Deutsch, Română', 'English, Bahasa indonesia, Bahasa melayu', 'English, Italiano, 日本語, Pусский', 'English, 日本語', 'Magyar, English', 'English, Deutsch, Español', 'Pусский', 'shqip, English, ', 'English, Deutsch, Français',

In [61]:
# Filtramos la columna spoken_languages solo las filas que contengan 'Español o English' 
filtered_df = df_movies[df_movies['spoken_languages'].str.contains('Español|English', na=False)]
filtered_df.head(2)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,release_year,actor,director,return
0,Toy Story Collection,30000000,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,English,Released,NaN,Toy Story,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",1995,"Tom Hanks, Tim Allen, Don Rickles",John Lasseter,12.451801
1,No collecction,65000000,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures Teitler Film Interscope Commu...,United States of America,1995-12-15,262797249.0,104.0,"English, Français",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",1995,"Robin Williams, Jonathan Hyde, Kirsten Dunst",Joe Johnston,4.043035


In [62]:
filtered_df.shape

(29832, 24)

In [63]:
print(filtered_df['release_year'].min())
print(filtered_df['release_year'].max())

1903
2020


In [ ]:
# Filtrar el DataFrame para obtener solo las películas con release_year mayor a 1980
filtered_df = filtered_df[filtered_df['release_year'] > 1980]

In [65]:
filtered_df.shape

(21875, 24)

In [66]:
filtered_df.head(2)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,release_year,actor,director,return
0,Toy Story Collection,30000000,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,English,Released,NaN,Toy Story,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",1995,"Tom Hanks, Tim Allen, Don Rickles",John Lasseter,12.451801
1,No collecction,65000000,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures Teitler Film Interscope Commu...,United States of America,1995-12-15,262797249.0,104.0,"English, Français",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",1995,"Robin Williams, Jonathan Hyde, Kirsten Dunst",Joe Johnston,4.043035


In [67]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21875 entries, 0 to 45537
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  21875 non-null  object        
 1   budget                 21875 non-null  int64         
 2   genres                 21875 non-null  object        
 3   id                     21875 non-null  int32         
 4   original_language      21875 non-null  object        
 5   overview               21875 non-null  object        
 6   popularity             21875 non-null  float64       
 7   production_companies   21875 non-null  object        
 8   production_countries   21875 non-null  object        
 9   release_date           21875 non-null  datetime64[ns]
 10  revenue                21875 non-null  float64       
 11  runtime                21853 non-null  float64       
 12  spoken_languages       21875 non-null  object        
 13  status

# 3. Carga/Disponibilización de datos (Load)

A fin de optimizar la etapa de análisis exploratorio, se han definido un conjunto de variables relevantes, entre las que se incluyen:'id', 'belongs_to_collection', 'budget', 'genres', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'title', 'vote_average', 'vote_count', 'release_year', 'actor', 'director', 'return'. Estoa datos han sido exportados a un archivo parquet, un formato eficiente para almacenar grandes volúmenes de datos estructurados.




In [68]:
columns_var = ['id','belongs_to_collection','budget','genres','overview','popularity','production_companies','production_countries','release_date','revenue','runtime','spoken_languages','title','vote_average','vote_count','release_year','actor','director','return']

In [69]:
df_limpio = filtered_df[columns_var]

In [70]:
df_limpio.head(2)

,id,belongs_to_collection,budget,genres,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,title,vote_average,vote_count,release_year,actor,director,return
0,862,Toy Story Collection,30000000,"Animation, Comedy, Family","Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,English,Toy Story,7.7,5415.0,1995,"Tom Hanks, Tim Allen, Don Rickles",John Lasseter,12.451801
1,8844,No collecction,65000000,"Adventure, Fantasy, Family",When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures Teitler Film Interscope Commu...,United States of America,1995-12-15,262797249.0,104.0,"English, Français",Jumanji,6.9,2413.0,1995,"Robin Williams, Jonathan Hyde, Kirsten Dunst",Joe Johnston,4.043035


In [71]:
df_limpio.shape

(21875, 19)

In [72]:
# Guardamos el DF en un formato parquet
df_limpio.to_parquet('D:/2024/HenryData/Py_Individual/PI_Recomendacion/Datasets/df_limpio.parquet', index=False)
#D:\2024\HenryData\Py_Individual\PI_Recomendacion\Datasets